# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core import Environment
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
import os
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.train.automl import AutoMLConfig
import pandas as pd
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()
experiment_name = 'Capstone-hyperdrive'

experiment=Experiment(ws, experiment_name)

found = False
key = "Precipitation-LA"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
        

if not found:
        # Gets a dataset from a given URL
        example_data = 'https://raw.githubusercontent.com/RaikohGrass/ML_Azure_Capstone/main/precipitation_LA.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        df = dataset.to_pandas_dataframe()
        df.describe()
        # We drop the empty column TAVG
        dataset = dataset.drop_columns('TAVG')
        # Since we are not using deep learning or techniques that have a notion of time we can drop the date and PGTM columns
        # We also drop the station name column
        dataset = dataset.drop_columns(['STATION','NAME','DATE','PGTM'])
        #Register the dataset in the default datastore
        dataset = dataset.register(workspace=ws,
                                   name=key)

In [15]:
df

,STATION,NAME,DATE,AWND,PGTM,PRCP,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT08
0,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2016-01-01,2.46,None,0.00,None,64,43,10.0,30.0,8.1,11.0,NaN,NaN,1.0
1,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2016-01-02,2.01,None,0.00,None,65,47,270.0,30.0,6.0,8.9,NaN,NaN,NaN
2,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2016-01-03,0.67,None,0.00,None,62,44,150.0,150.0,10.1,14.1,NaN,NaN,NaN
3,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2016-01-04,1.34,None,0.01,None,69,55,270.0,280.0,8.1,14.1,NaN,NaN,NaN
4,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2016-01-05,2.46,None,1.61,None,59,49,140.0,140.0,10.1,16.1,1.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2020-12-27,1.12,None,0.01,None,66,55,270.0,260.0,8.9,18.1,1.0,NaN,1.0
1823,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2020-12-28,4.70,None,1.81,None,56,47,90.0,260.0,14.1,21.0,1.0,NaN,NaN
1824,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2020-12-29,1.57,None,0.00,None,65,42,340.0,360.0,10.1,18.1,NaN,NaN,NaN
1825,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2020-12-30,0.45,None,0.00,None,69,44,260.0,260.0,6.9,12.1,NaN,NaN,NaN


In [13]:
path ="https://raw.githubusercontent.com/RaikohGrass/ML_Azure_Capstone/main/precipitation_LA.csv"

ds = TabularDatasetFactory.from_delimited_files(path)

In [49]:
from sklearn.model_selection import train_test_split
df = ds.to_pandas_dataframe()
# We drop the empty column TAVG
df = df.drop('TAVG',1)
# Since we are not using deep learning or techniques that have a notion of time we can drop the date and PGTM columns
# We also drop the station name column
df = df.drop(['STATION','NAME','DATE','PGTM'],1)
x = df.fillna(0)
y = x.pop('PRCP')

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [50]:
x_train

,AWND,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT08
529,2.68,81,60,270.0,270.0,12.1,17.0,0.0,0.0,0.0
1675,0.89,88,66,260.0,270.0,8.1,13.0,1.0,0.0,1.0
1727,0.89,85,63,260.0,260.0,8.1,13.0,1.0,0.0,1.0
988,1.57,86,65,270.0,270.0,8.9,14.1,0.0,0.0,0.0
182,2.46,80,66,260.0,260.0,8.1,14.1,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
1130,2.24,60,51,260.0,270.0,8.1,15.0,1.0,0.0,0.0
1294,2.24,82,65,270.0,270.0,8.9,15.0,0.0,0.0,1.0
860,1.34,76,60,270.0,260.0,8.9,16.1,0.0,0.0,1.0
1459,2.24,66,54,60.0,60.0,8.1,13.0,0.0,0.0,0.0


In [ ]:
param_sampling = RandomParameterSampling({'--criterion':choice('squared_error','friedman_mse','absolute_error','poisson'),
                                          '--max_depth':choice(2,4,6,8,10),
                                          '--min_samples_split':choice(2,4,6,8,10),
                                          '--max_leaf_nodes':choice(4,8,12,16,20)})
hyperparameters : {"--criterion": "absolute_error", "--max_depth": 6, "--max_leaf_nodes": 8, "--min_samples_split": 8

In [52]:
model = DecisionTreeRegressor(criterion = 'absolute_error').fit(x_train, y_train)

KeyError: 'absolute_error'

In [53]:
model = DecisionTreeRegressor(criterion = 'absolute_error',
                              max_depth = 6,
                              min_samples_split = 8,
                              max_leaf_nodes = 8).fit(x_train, y_train)


KeyError: 'absolute_error'

In [23]:
x_train

,AWND,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT08
529,2.68,81,60,270.0,270.0,12.1,17.0,NaN,NaN,NaN
1675,0.89,88,66,260.0,270.0,8.1,13.0,1.0,NaN,1.0
1727,0.89,85,63,260.0,260.0,8.1,13.0,1.0,NaN,1.0
988,1.57,86,65,270.0,270.0,8.9,14.1,NaN,NaN,NaN
182,2.46,80,66,260.0,260.0,8.1,14.1,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...
1130,2.24,60,51,260.0,270.0,8.1,15.0,1.0,NaN,NaN
1294,2.24,82,65,270.0,270.0,8.9,15.0,NaN,NaN,1.0
860,1.34,76,60,270.0,260.0,8.9,16.1,NaN,NaN,1.0
1459,2.24,66,54,60.0,60.0,8.1,13.0,NaN,NaN,NaN


In [4]:
cpu_cluster_name = 'EduardoCluster'
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [24]:
sklearn_env = Environment.get(workspace=ws, name="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu")
from azureml.core import ScriptRunConfig
src = ScriptRunConfig(source_directory='./',compute_target=cpu_cluster_name,script='train.py',environment=sklearn_env)


# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({'--criterion':choice('squared_error','friedman_mse','absolute_error','poisson'),
                                          '--max_depth':choice(2,4,6,8,10),
                                          '--min_samples_split':choice(2,4,6,8,10),
                                          '--max_leaf_nodes':choice(4,8,12,16,20)})



#TODO: Create your estimator and hyperdrive config
estimator = src

hyperdrive_run_config = HyperDriveConfig(run_config=estimator,
                                         policy=early_termination_policy,
                                         hyperparameter_sampling=param_sampling,
                                         primary_metric_name='Accuracy',
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                         max_total_runs=12)

In [25]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [26]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

